# Import and export data.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import dataset.

In [ ]:
# Files
dname = '/media/jacques/DATA/2019_MusicPred/experimentContinuousMatrix/data/behavior/'
fnames = glob.glob(dname + '*.xpd')

# Import dataframe
var_names = ('suj', 'condition', 'i_seq', 'bloc', 'trial', 'i_tone', 'decision', 'resp', 'rt', 'accurate')
DF = pd.DataFrame(columns=var_names)
for i, fname in enumerate(fnames):
    data = np.loadtxt(fname, skiprows=12, delimiter=',', 
                      dtype={'formats': ('i4', 'S12', 'i4', 'i4', 'i4', 'i4', 'S10', 'S10', 'S10', 'S10'), 
                             'names': var_names})
    _df = pd.DataFrame(data, columns=var_names)
    DF = DF.append(_df, ignore_index=True)

In [ ]:
DF = DF.loc[DF['condition'] == 'random']

In [ ]:
DF.loc[DF['decision'] == 'True', 'decision'] = True
DF.loc[DF['decision'] == 'None', 'decision'] = False
DF.loc[DF['resp'] == 'None', 'resp'] = np.nan
DF.loc[DF['rt'] == 'None', 'rt'] = np.nan

### Get dimensions.

In [ ]:
sujs = DF['suj'].unique()
blocs = DF['bloc'].unique()
i_seqs = DF['i_seq'].unique()
n_sujs = sujs.size
n_blocs = blocs.size
n_seqs = i_seqs.size
N_VOC = DF['i_tone'].unique().size
n_tones = DF['trial'].unique().size
n_dec = np.sum(DF.loc[DF['suj'] == sujs[-1], 'decision']) / n_seqs
n_sujs, n_blocs, n_seqs, n_tones, N_VOC, n_dec

### Build arrays of data.

In [ ]:
CHOICES = np.full((n_sujs, n_dec), np.nan) # Choice made by the participant
SEQS = np.full((n_sujs, n_tones), np.nan) # The sequence of played tones
PERF = np.full((n_sujs, n_dec), np.nan) # Whether the participant made a correct choice or not
RT = np.full((n_sujs, n_dec), np.nan) # Reaction time
SEQS.shape

In [ ]:
for s, suj in enumerate(sujs):

    # Retrieve sequence
    seq = np.array(DF.loc[DF['suj'] == suj, 'i_tone'], dtype=np.int)
    if seq.size == 0: continue

    # Sequence
    SEQS[s] = seq.copy()

    # Choices
    choices = np.array(DF.loc[DF['suj'] == suj, 'resp'], dtype=np.float)
    CHOICES[s] = choices.copy()

    # Response time
    RT[s] = np.array(DF.loc[DF['suj'] == suj, 'rt'], dtype=np.float)

    # Performance
    if seq.size == 0: continue
    PERF[s] = seq == choices
    PERF[s][np.isnan(choices)] = np.nan

### Save all arrays.

In [ ]:
np.save(dname + 'CHOICES_TRAINING', CHOICES)
np.save(dname + 'SEQS_TRAINING', SEQS)
np.save(dname + 'PERF_TRAINING', PERF)
np.save(dname + 'RT_TRAINING', RT)